#Harry Winston Project

##Import library

In [9]:
from bs4 import BeautifulSoup # For HTML parsing
import requests # Website connections
from time import sleep # To prevent overwhelming the server between connections
import pandas as pd # For converting results to a dataframe and bar chart plots
from selenium import webdriver
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By

In [59]:
#Harry Winston website
page_url = 'https://www.harrywinston.com/en/timepieces'

In [60]:
# Fetches the content of the given URL using a GET request and parses it as HTML
result = requests.get(page_url)
soup = BeautifulSoup(result.content, "html.parser")

##Collect links collection

In [61]:
# Initialize an empty list to store extracted links
links_collection  = []
# Find all elements with the class 'collection-wrapper' in the parsed HTML
for collection in soup.find_all('div', class_= 'collection-wrapper'):
    link_element = collection.find('a')
    #print('link_element',link_element)

    if link_element:
      link = 'https://www.harrywinston.com'+ link_element["href"]
      links_collection.append(link)
      #print('links_collection',links_collection)

links_collection

['https://www.harrywinston.com/en/products/high-jewelry-timepieces-by-harry-winston',
 'https://www.harrywinston.com/en/products/harry-winston-emerald-collection',
 'https://www.harrywinston.com/en/products/harry-winston-the-ocean-collection',
 'https://www.harrywinston.com/en/products/harry-winston-the-avenue-collection',
 'https://www.harrywinston.com/en/products/harry-winston-the-premier-collection',
 'https://www.harrywinston.com/en/products/harry-winston-midnight-collection',
 'https://www.harrywinston.com/en/products/histoire-de-tourbillon-and-opus']

In [40]:
# Create an empty DataFrame with columns representing watch information
df = pd.DataFrame(columns=['reference_number','watch_URL','type','brand','year_introduced','parent_model','specific_model','nickname',
'marketing_name','style','currency','price','image_URL','made_in','case_shape','case_material','case_finish','caseback','diameter',
'between_lugs','lug_to_lug','case_thickness','bezel_material','bezel_color', 'crystal','water_resistance','weight','dial_color',
'numerals','bracelet_material','bracelet_color','clasp_type','movement','caliber','power_reserve','frequency','jewels','features',
'description','short_description'])

df

,reference_number,watch_URL,type,brand,year_introduced,parent_model,specific_model,nickname,marketing_name,style,...,bracelet_color,clasp_type,movement,caliber,power_reserve,frequency,jewels,features,description,short_description


## Collect watch information





In [54]:
def collect_watch_info(url):
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    isLimited = False
    case_material = None
    diameter = None
    case_thickness = None
    caseback = None
    water_resistance = None
    movement = None
    caliber = None
    features = None
    dial_color = ''
    bracelet_material = None
    bracelet_color = ''
    clasp_type = None
    short_description = None # Some watches doesn't have a short description        

    specifications_lst = soup.find_all('div', class_='pdp-timepiece-specifications__specs-section')

    for j in range(len(specifications_lst)): #Technical specifications
        if specifications_lst[j].find('h4').text == 'Case':
            case_lst = specifications_lst[j]
            divs = case_lst.find_all('div', class_='pdp-timepiece-specifications__specs-section-item')

            for i in range(len(divs)):
                if divs[i].find('h5').text == 'Material':
                    case_material = divs[i].find('p').text
                elif divs[i].find('h5').text == 'Dimensions':
                    diameter = divs[i].find('p').text
                elif divs[i].find('h5').text == 'Height':
                    case_thickness = divs[i].find('p').text
                elif divs[i].find('h5').text == 'Caseback':
                    caseback = divs[i].find('p').text
                elif divs[i].find('h5').text == 'Water-resistance (bar)':
                    water_resistance = divs[i].find('p').text + " bar"

        elif specifications_lst[j].find('h4').text == 'Movement':
            movement_lst = specifications_lst[j]
            divs = movement_lst.find_all('div', class_='pdp-timepiece-specifications__specs-section-item')

            for i in range(len(divs)):
                if divs[i].find('h5').text == 'Type':
                    movement = divs[i].find('p').text
                elif divs[i].find('h5').text == 'Caliber':
                    caliber = divs[i].find('p').text
                elif divs[i].find('h5').text == 'Functions':
                    features = divs[i].find('div').text.strip().replace('\n',' ')

        elif specifications_lst[j].find('h4').text == 'Dial':
            dial_lst = specifications_lst[j]
            divs = dial_lst.find_all('div', class_='pdp-timepiece-specifications__specs-section-item')

            for i in range(len(divs)):
                dial_color = dial_color + " ".join(divs[i].text.split()) + " "

        elif specifications_lst[j].find('h4').text == 'Bracelet':
            bracelet_lst = specifications_lst[j]
            divs = bracelet_lst.find_all('div', class_='pdp-timepiece-specifications__specs-section-item')

            for i in range(len(divs)): 
                if divs[i].find('h5').text == 'Material': # Listed under "Technical Specifications" > "Bracelet" > Material AND Gem-setting fields
                    bracelet_material = " ".join(divs[i].text.split()) + " "
                    bracelet_color = bracelet_material
                elif divs[i].find('h5').text == 'Gem-setting': # Listed under "Technical Specifications" > "Bracelet" > Material AND Gem-setting fields
                    bracelet_color = bracelet_color + " " + " ".join(divs[i].text.split()) + " "
                    bracelet_material = bracelet_color
                elif divs[i].find('h5').text == 'Buckle': # Listed under "Technical Specifications" > "Bracelet" > Buckle
                    clasp_type = divs[i].find('div').text.strip()

        elif specifications_lst[j].find('h4').text == 'General':
            general_lst = specifications_lst[j]
            divs = general_lst.find_all('div', class_='pdp-timepiece-specifications__specs-section-item')

            
            
            for i in range(len(divs)): 
                if 'gem-setting' in divs[i].find('h5').text: # Pull gem-setting info under "General"
                    short_description = " ".join(divs[i].find('div').text.split()) + " "
                elif divs[i].find('h5').text == 'Limited edition': # isLimited
                    isLimited = True

    # ---------------------------------------------------------------- driver ----------------------------------------------------------------
    driver = Firefox()
    driver.get(url)

    # watch_URL
    try:
        watch_URL = url
    except Exception:
        watch_URL = 'N/A'
    
    # parent_model
    try:
        parent_model = driver.find_element(By.XPATH, '/html/body/div[2]/main/div/div[1]/div/div/div/div[2]/div/div/p/a').text
        if 'HW Harry Winston' in parent_model:
            parent_model = parent_model.replace('HW Harry Winston', '').strip()
        elif 'by Harry Winston' in parent_model:
            parent_model = parent_model.replace('by Harry Winston', '').strip()
        elif 'Harry Winston ' in parent_model:
            parent_model = parent_model.replace('Harry Winston', '').strip()

    except Exception:
        parent_model = 'N/A'

    # specific_model
    try:
        specific_model = driver.find_element(By.XPATH, '//*[@id="hw-content"]/div[1]/div/div/div/div[2]/div/div/h1').text
    except Exception:
        specific_model = 'N/A'

    # nickname
    try:
        nickname = driver.find_element(By.XPATH, '//*[@id="hw-content"]/div[1]/div/div/div/div[2]/div/div/h1').text
    except Exception:
        nickname = 'N/A'

    # image_URL
    try:
        image_URL = soup.find_all('ul', class_='glide__slides')[0].find('img').get('src')
    except Exception:
        image_URL = 'N/A'
    
    
    if isLimited: # ---------------------------------------------------------------- Limited ----------------------------------------------------------------
        # reference_number 
        try:
            reference_number = driver.find_element(By.XPATH, '//*[@id="hw-content"]/div[1]/div/div/div/div[2]/div/div/div[4]/div/p').text.split()[2]
        except Exception:
            reference_number = 'N/A'

        # marketing_name 
        try:
            marketing_name =  driver.find_element(By.XPATH, '/html/body/div[2]/main/div/div[1]/div/div/div/div[2]/div/div/div[1]/p').text
        except Exception:
            marketing_name = 'N/A'
            
        # description 
        try:
            description = driver.find_element(By.XPATH, '//*[@id="hw-content"]/div[1]/div/div/div/div[2]/div/div/div[4]/p').text
        except Exception:
            description = 'N/A'
    else: # ---------------------------------------------------------------- Not Limited ----------------------------------------------------------------
        # reference_number 
        try:
            reference_number = driver.find_element(By.XPATH, '//*[@id="hw-content"]/div[1]/div/div/div/div[2]/div/div/div[3]/div').text.split()[2]
        except Exception:
            reference_number = 'N/A'

        marketing_name = None #  Not Limited Edition
        
        # description
        try:
            description = driver.find_element(By.XPATH, '//*[@id="hw-content"]/div[1]/div/div/div/div[2]/div/div/div[3]/p').text
        except Exception:
            description = 'N/A'
    
    watch_info = {
        'reference_number' : reference_number,
        'watch_URL' : watch_URL,
        'type' : None, 
        'brand' : 'Harry Winston' ,
        'year_introduced' : None,
        'parent_model' : parent_model,
        'specific_model' : specific_model,
        'nickname' : nickname,
        'marketing_name' : marketing_name,
        'style' : None,
        'currency' : None,
        'price' : None,
        'image_URL' : image_URL,
        'made_in' : None,
        'case_shape' : None,
        'case_material' : case_material,
        'case_finish' : None,
        'caseback' : caseback,
        'diameter' : diameter,
        'between_lugs' : None,
        'lug_to_lug' : None,
        'case_thickness' : case_thickness,
        'bezel_material' : None,
        'bezel_color' : None,
        'crystal' : None,
        'water_resistance' : water_resistance,
        'weight' : None,
        'dial_color' : dial_color,
        'numerals' : None,
        'bracelet_material' : bracelet_material,
        'bracelet_color' : bracelet_color,
        'clasp_type' : clasp_type,
        'movement' : movement,
        'caliber' : caliber,
        'power_reserve' : None,
        'frequency' : None,
        'jewels' : None,
        'features' : features,
        'description' : description,
        'short_description' : short_description
    }
    driver.close()
    return watch_info

In [48]:

for link in links_collection:
    # Create a new Firefox WebDriver instance for each link
    driver = Firefox()
    # Navigate to the watch page using the extracted link
    driver.get(link)
     # Locate the element (assumed to be an input button) using XPATH
    input_btn = driver.find_element(By.XPATH, "/html/body/div[2]/main/div/div[2]/div/div[5]/div/input")
    while True:
        try:
            input_btn.click()
            sleep(5)
        except:
            break
    # Parse the HTML content of the current page
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    links_watches  = []
    for collection in soup.find('ul', class_='search-result-list').find_all('li'):
        # Extract the link to the individual watch page
        link_watch = collection.find('a')

        if link_watch:
            link = 'https://www.harrywinston.com'+ link_watch["href"]
            links_watches.append(link)
             # Fetch the HTML content of the individual watch page
            # soup = BeautifulSoup(requests.get(link).content, "html.parser")

            # Extract watch information using   collect_watch_info function
            new_row_data = collect_watch_info(link)
            df.loc[len(df)] = new_row_data
            
    driver.close()

df

,reference_number,watch_URL,type,brand,year_introduced,parent_model,specific_model,nickname,marketing_name,style,...,bracelet_color,clasp_type,movement,caliber,power_reserve,frequency,jewels,features,description,short_description
0,OCEABI42ZZ004,https://www.harrywinston.com/en/products/harry...,None,Harry Winston,None,The Ocean Collection,Zalium Variation Turquoise/Orange,Zalium Variation Turquoise/Orange,Zalium Variation Turquoise/Orange,None,...,Material turquoise alligator leather with oran...,Zalium,automatic,HW3305,None,None,None,excentered hours and minutes retrograde second...,The Ocean Collection presents three models tha...,None
1,OCEABI42ZZ005,https://www.harrywinston.com/en/products/harry...,None,Harry Winston,None,The Ocean Collection,Zalium Variation Neon Green/Black,Zalium Variation Neon Green/Black,Zalium Variation Neon Green/Black,None,...,Material neon green alligator leather with bla...,Zalium,automatic,HW3305,None,None,None,excentered hours and minutes retrograde second...,The Ocean Collection presents three models tha...,None
2,OCEABI42ZZ006,https://www.harrywinston.com/en/products/harry...,None,Harry Winston,None,The Ocean Collection,Zalium Variation White/Navy Blue,Zalium Variation White/Navy Blue,Zalium Variation White/Navy Blue,None,...,Material white alligator leather with navy blu...,Zalium,automatic,HW3305,None,None,None,excentered hours and minutes retrograde second...,The Ocean Collection presents three models tha...,None
3,OCEAMP42WW002,https://www.harrywinston.com/en/products/harry...,None,Harry Winston,None,The Ocean Collection,Ocean Date Moon Phase Automatic 42mm,Ocean Date Moon Phase Automatic 42mm,None,None,...,Material blue alligator leather Gem-setting 4...,18K white gold,automatic,HW3203,None,None,None,excentered hours and minutes date moon phase,The House of Harry Winston unveils an Ocean ti...,352 brilliant-cut diamonds (approx. 4.45 cts)
4,OCEABI36WW057,https://www.harrywinston.com/en/products/harry...,None,Harry Winston,None,The Ocean Collection,Ocean Biretrograde Automatic 36mm,Ocean Biretrograde Automatic 36mm,None,None,...,Material white pearl alligator leather Gem-se...,18K white gold,automatic,HW3302,None,None,None,excentered hours and minutes retrograde second...,Harry Winston’s Ocean timepiece collection wel...,198 brilliant-cut diamonds (approx. 3.55 cts)1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,HCOMDT51WW004,https://www.harrywinston.com/en/products/histo...,None,Harry Winston,None,Histoire de Tourbillon & Opus,Histoire de Tourbillon 8,Histoire de Tourbillon 8,Histoire de Tourbillon 8,None,...,Material black alligator leather,18K white gold,manual,HW4503,None,None,None,excentered hours and minutes power reserve,The House of Harry Winston continues to explor...,None
422,HCOMDT51PP001,https://www.harrywinston.com/en/products/histo...,None,Harry Winston,None,Histoire de Tourbillon & Opus,Histoire de Tourbillon 7,Histoire de Tourbillon 7,Histoire de Tourbillon 7,None,...,Material black alligator leather,platinum,manual,HW4502,None,None,None,excentered hours and minutes power reserve,"Featuring two biaxial tourbillons, Histoire de...",None
423,HCOMDT51WW001,https://www.harrywinston.com/en/products/histo...,None,Harry Winston,None,Histoire de Tourbillon & Opus,Histoire de Tourbillon 7,Histoire de Tourbillon 7,Histoire de Tourbillon 7,None,...,Material black alligator leather,18K white gold,manual,HW4502,None,None,None,excentered hours and minutes power reserve,"Featuring two biaxial tourbillons, Histoire de...",None
424,HCOMDT51WW002,https://www.harrywinston.com/en/products/histo...,None,Harry Winston,None,Histoire de Tourbillon & Opus,Histoire de Tourbillon 7,Histoire de Tourbillon 7,Histoire de Tourbillon 7,None,...,Material black alligator leather,18K white gold,manual,HW4502,None,None,None,excentered hours and minutes power reserve,"Featuring two biaxial tourbillons, Histoire de...",None


## Save the DataFrame to a CSV file

In [49]:
df.to_csv('harrywinston_data_27feb.csv', index=False)